# String
<hr>

### 344 Reverse String

使用双指针法来进行reverse。要求$O(1)$.首位交换，再往中间移动再换。



In [5]:
from typing import List
class Solution:
    def reverseString(self, s: List[str]) -> None:
        """
        Do not return anything, modify s in-place instead.
        """
        # define the length of the string
        n = len(s)

        # define two pointers
        left, right = 0, n-1

        # loop until the two pointers meet
        while left < right:
            # swap the two elements
            s[left], s[right] = s[right], s[left]
            # move the two pointers
            left, right = left+1, right-1

# test the solution
s = Solution()
test1 = ["h","e","l","l","o"]
s.reverseString(test1)
print(test1)


['o', 'l', 'l', 'e', 'h']


### 541. Reverse String II
每次移动pointer的时候，不用`i++` `i += 2k`来移动就好了。

就是在`for`循环的时候，用`for i in range(0, n, 2 * k)`

其次进行swap的时候，用`s[i: i+k] = swap(s[i: i+k])`

In [3]:
class Solution:
    def reverseStr(self, s: str, k: int) -> str:
        def swap(text: list) -> list:
            n = len(text)
            left, right = 0, n-1
            while left < right:
                text[left], text[right] = text[right], text[left]
                left, right = left+1, right-1
            return text
        
        s = list(s)
        n = len(s)

        for i in range(0, n, 2*k):
            s[i:i+k] = swap(s[i:i+k])

        return "".join(s)
    
# test the solution
s = Solution()
test = "abcdefg"
print(s.reverseStr(test, 2))

         

bacdfeg
